# Exercise 2
Today we are going to continue to work on point clouds.
We will work on clustering point clouds. That enables us to segment them.

In [1]:
import copy

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
from sklearn.cluster import KMeans

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def draw_labels_on_model(pcl, labels):
    cmap = plt.get_cmap("tab20")
    pcl_temp = copy.deepcopy(pcl)
    max_label = labels.max()
    print("%s has %d clusters" % (pcl_name, max_label + 1))
    colors = cmap(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    pcl_temp.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.visualization.draw_geometries([pcl_temp])



## K-means on a cube
We created a point cloud using `open3d`.
Our goal is to segment each side using k-means.

In [3]:
pcl_name = 'Cube'
density = 1e4  # density of sample points to create
pcl = o3d.geometry.TriangleMesh.create_box().sample_points_uniformly(int(density))
eps = 0.4
print("%s has %d points" % (pcl_name, np.asarray(pcl.points).shape[0]))
o3d.visualization.draw_geometries([pcl])

Cube has 10000 points


If we just use k-means out of the box with the point cloud, we will get what just has been visualized.

Note: Using the '+' and '-' keys in the viewer will increase/decrease the size of the points.

In [4]:
km = KMeans(n_clusters=6, init='random',
            n_init=10, max_iter=300, tol=1e-04, random_state=0)

# Get the points from the pointcloud as nparray
xyz = np.asarray(pcl.points)
labels = km.fit_predict(xyz)
draw_labels_on_model(pcl, labels)

Cube has 6 clusters


We can see that we get six clusters, but they do not span a side.

We try again, but this time we instead use the normals of the cube as input for k-means.

The normals for each plane should be parallel with the other normals from said plane.

In [7]:
pcl.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
normals = np.asarray(pcl.normals)

km_normals = KMeans(n_clusters=6, init='random', n_init=10, max_iter=300, tol=1e-04, random_state=0)

normal_labels = km_normals.fit_predict(normals)
draw_labels_on_model(pcl, normal_labels)

Cube has 6 clusters


This still does not work, opposite sides will also have normals that point the other way ($\vec{n}$ and $-\vec{n}$).

So, to combat this we can attempt to use the xyz coordinates and the normals.

## More exercises

### A) K-means continued.

Combine the point cloud points (xyz) with the normals and do k-means.

```xyz_n = np.concatenate((xyz, normals), axis=1)```

Do you get better clusters?
Why would adding the normals help?

In [6]:

km = KMeans(n_clusters=6, init='random',
            n_init=10, max_iter=300, tol=1e-04, random_state=0)

xyz_n = np.concatenate((xyz, normals), axis=1)
labels = km.fit_predict(xyz_n)
draw_labels_on_model(pcl, labels)

Cube has 6 clusters



### B) 
Try weighting either the points or normals by scaling them by some factor. Can this perfectly segment each of the faces of the cube?


In [12]:
scale_factor_points = 2.5
scale_factor_normals = 1.

xyz_n = np.concatenate((xyz * scale_factor_points, normals * scale_factor_normals), axis=1)
km = KMeans(n_clusters=6, init='random',
            n_init=10, max_iter=300, tol=1e-04, random_state=0)
labels = km.fit_predict(xyz_n)
draw_labels_on_model(pcl, labels)

Cube has 6 clusters


### C)
Try to cluster all the different shapes using k means.
```{Python}
d = 4
mesh = o3d.geometry.TriangleMesh.create_tetrahedron().translate((-d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_octahedron().translate((0, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_icosahedron().translate((d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_torus().translate((-d, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_moebius(twists=1).translate(
    (0, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_moebius(twists=2).translate(
    (d, -d, 0))
mesh.sample_points_uniformly(int(1e5)), 0.5
```


In [18]:
d = 4
mesh = o3d.geometry.TriangleMesh().create_tetrahedron().translate((-d, 0, 0))
mesh += o3d.geometry.TriangleMesh().create_octahedron().translate((0, 0, 0))
mesh += o3d.geometry.TriangleMesh().create_icosahedron().translate((d, 0, 0))
mesh += o3d.geometry.TriangleMesh().create_torus().translate((-d, -d, 0))

# 'open3d.cuda.pybind.geometry.TriangleMesh' object has no attribute 'create_moebius'
# mesh += o3d.geometry.TriangleMesh().create_moebius(twists=1).translate((0, -d, 0))
# mesh += o3d.geometry.TriangleMesh().create_moebius(twists=2).translate((d, -d, 0))
pcl = mesh.sample_points_uniformly(int(1e5))

xyz = np.asarray(pcl.points)

km = KMeans(n_clusters=4, init='random', n_init=10, max_iter=300, tol=1e-04, random_state=0)
labels = km.fit_predict(xyz)
draw_labels_on_model(pcl, labels)

Cube has 4 clusters



### D)
Now try segmenting a different point cloud located at `pointclouds/fragment.ply`
Are you able to cluster the point cloud?

Which features could be useful to segment this point cloud?
- fpfh features?
- xyz
- normals 
- colors

Are you able to get clusters that make sense? Why?


In [32]:
# load TestData/fragment.ply
pcd = o3d.io.read_point_cloud("TestData/fragment.ply")

pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
normals = np.asarray(pcd.normals)
xyz = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)

a, b, c = 0.3,2.,10.

data = np.concatenate([a * xyz, b * normals, c * colors], axis=1)

km = KMeans(n_clusters=20, init='random',
            n_init=10, max_iter=300, tol=1e-04, random_state=0)

# Get the points from the pointcloud as nparray

labels = km.fit_predict(xyz)
draw_labels_on_model(pcd, labels)

Cube has 20 clusters



### E)
Use the built-in `cluster_dbscan` algorithm.
Tweak the parameters and see what you get out.

Attempt on the combined figures and on `fragment.ply`
```{Python}
#eps (float) – Density parameter that is used to find neighbouring points.
eps = 0.02

#min_points (int) – Minimum number of points to form a cluster.
min_points = 10

labels = np.array(pcl.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))
```

In [35]:
pcd = o3d.io.read_point_cloud("TestData/fragment.ply")

eps = 0.02
min_points = 10

labels = np.array(pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))
draw_labels_on_model(pcd, labels)

Precompute neighbors.[========================================] 100%
Clustering[========================================] 100%
Precompute neighbors.[========================================] 100%
Clustering[========================================] 100%
Precompute neighbors.[========================================] 100%
